In [1]:
from bs4 import BeautifulSoup
import csv
import requests
import pandas as pd
import  os 

In [2]:
def initializing_page(url):
    req = requests.get(url).text
    soup = BeautifulSoup(req,'html.parser')
    body = soup.body
    return body

In [3]:
def Grap_categories(url):
    body = initializing_page(url)
    ul = body.find('ul',attrs = {'class': 'nav nav-list'})
    li = ul.find('li').find('ul').find_all('li')
    categories = {i.text.strip():('https://books.toscrape.com/'+ i.find('a').attrs['href']) for i in li }
    return categories

In [4]:
def grap_page_books(url):
    numbers = {'One':1,'Two':2,'Three':3, 'Four':4,'Five':5}
    body = initializing_page(url)
    all_books = body.find_all('li',attrs = {'class': 'col-xs-6 col-sm-4 col-md-3 col-lg-3'})
    books_names =[i.find('h3').find('a').attrs['title'].replace('â\x80\x99','\'') for i in all_books]
    rates = [numbers[i.find('p').attrs['class'][-1]] for i in all_books]
    price = [float(i.find('p',attrs={'class': 'price_color'}).text.split('£')[-1]) for i in all_books]
    return list(zip(books_names,price,rates))

In [5]:
def no_books_inpage(url):
    body = initializing_page(url)
    result = int(body.find('form',{'method': 'get','class':'form-horizontal'}).find('strong').text)
    return result

In [63]:
def main():
    urls = list(Grap_categories('https://books.toscrape.com/').values())
    categories = list(Grap_categories('https://books.toscrape.com/').keys())
    for i in range(len(urls)):
        li1 = grap_page_books(urls[i])
        for y in range(len(li1)):
            li2 = list(li1[y])
            li2.append(categories[i])
            if 'books.csv'not in os.listdir():
                with open('books.csv','w',newline='',encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow(['name', 'price', 'rate', 'category_name'])
                    writer.writerow(li2)
            else:
                with open('books.csv','a',newline='',encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow(li2)
        if no_books_inpage(urls[i]) > 20 :
            for j in range(2,(no_books_inpage(urls[i]) // 20) + 2 ):
                li3 = grap_page_books(urls[i].replace('index',f'page-{j}'))
                for z in range(len(li3)):
                        li4 = list(li3[z])
                        li4.append(categories[i])
                        with open('books.csv','a',newline='',encoding="utf-8") as f:
                            writer = csv.writer(f)
                            writer.writerow(li4)    

In [64]:
main()

In [4]:
df = pd.read_csv('books.csv')

In [5]:
df.to_excel('books.xlsx',index=None)